# Hyperparameter Tuning for ESN on Kepler Time Series

## 1. Problem Statement

In our first experiment with a small Echo State Network (ESN) on the Kepler labelled dataset, the results showed:

- **Accuracy:** 99%
- **AUC:** 0.7
- **Errors:** 5 out of 570  

At first sight, this seems excellent. However, **all errors correspond to stars that actually have exoplanets**, which means the model **never detected any exoplanet**.  

**Problem:** The ESN is biased toward the majority class (no exoplanet) due to the extreme class imbalance in the dataset. The goal of this notebook is to **fine-tune the ESN hyperparameters** to improve detection of exoplanets.

## 2. Initial Setup

Here are the base parameters used previously:

```python
units = 20          # number of neurons in the reservoir
leak_rate = 0.3     # leak rate (memory retention of neurons)
input_scaling = 0.5 # influence of input on the reservoir
spectral_radius = 0.9 # strength of reservoir dynamics
ridge = 1e-6        # regularization
```

## 3. Parameters to Optimize

Based on the tutorial and our observations, we can focus on the following hyperparameters:

- ``units`` = Number of neurons in the reservoir, from 20 to 200

- ``leak_rate`` = Memory of the reservoir, from 0.1 to 1

- ``input_scaling`` = Influence of input on reservoir, from 0.1 to 1

- ``spectral_radius`` = Strength of dynamics, from 0.5 to 1.5

- ``ridge`` = Regularization (ridge), from 1e-8 to 1e-2


**Goal:** increase the network's ability to detect the minority class (exoplanets) without destabilizing the reservoir.

## 4. Strategy

1. Fix the random seed to ensure reproducibility.

2. Normalize each series individually (mean 0, std 1).

3. Define a grid or random search over the above hyperparameters.

4. For each configuration:

    - Train the ESN

    - Compute the AUC

5. Select hyperparameters that maximize AUC (more informative than accuracy due to imbalance) and minimize the number of errors.

## 5. Load Dataset and Preprocessing

This part is similar to the loading made in the base experiment.

In [1]:
import numpy as np
import pandas as pd

# Load your train/test datasets
train_df = pd.read_csv("../data/exoTrain.csv")
test_df = pd.read_csv("../data/exoTest.csv")

# Extract labels
y_train_np = np.asarray(train_df.iloc[:,0], dtype=int) - 1  # 0 = no exoplanet, 1 = exoplanet
y_test_np  = np.asarray(test_df.iloc[:,0], dtype=int) - 1

# Extract time series data
X_train = train_df.iloc[:, 1:].values[..., np.newaxis]
X_test  = test_df.iloc[:, 1:].values[..., np.newaxis]

# Convert to list of sequences
X_train_seq = [x for x in X_train]
X_test_seq  = [x for x in X_test]

# Normalize per series
X_train_seq = [(x - x.mean()) / (x.std() + 1e-8) for x in X_train_seq]
X_test_seq  = [(x - x.mean()) / (x.std() + 1e-8) for x in X_test_seq]

# Create sequential labels for ESN
y_train_seq = [np.full((x.shape[0], 1), y) for x, y in zip(X_train_seq, y_train_np)]



## 6. Hyperparameter Fine-tuning

Now that the ESN is all set up, we can start the fine-tuning experiment.

### Methodology 

The methodology we propose here is inspired by the paper *[Which Hype for my New Task? Hints and Random Search for Echo State Networks Hyperparameters](https://inria.hal.science/hal-03203318v2/document)*, published in 2021 by Xavier Hinaut and Nathan Trouvain. In this paper, the authors suggest that random search is more efficient than grid search for ESN hyperparameters optimization.

Thus, in the following steps, we will finetune hyperparameters thanks to a random search, instead of a grid search.

### Why random search?

The paper written by Xavier Hinaut and Nathan Trouvain explains that in a grid search, evaluations can be wasted in **low-impact regions**. Indeed, in reservoir computing, overall performance on hyperparameters is rather **non-linear and irregular**. There are a lot of chance that gird-search only sees low-impact regions because of these non-lineariries.

The advantage of random search is that it **samples uniformly** across the full range of each parameter. Hence, it covers more independent combinations and is more likely to find effective configurations with **fewer trials**. Because only a few hyperparameters strongly influence performance, random search spreads the search more efficiently in order to explore these influential directions. This approach reduces the redundancy inherent in grids and achieves better performance for equivalent search conditions.


### Defining the objective

The main objectives we aim to follow is to enhance the AUC, assure non-trivial detection of the minority class and avoid models which always predict "non-exoplanet star".

This can be done by maximizing the AUC score and enhancing the exoplanet class recall (sensibility).

#### Candidates Generation

A search space is defined, in which random combinations are made thanks to a selection on a uniform law, for each hyperparameter to optimize.

In [ ]:
def sample_hyperparameters(n_samples, seed=42):
    rng = np.random.default_rng(seed)
    configs = []

    for _ in range(n_samples):
        config = {
            "units": rng.integers(5, 20),
            "spectral_radius": rng.uniform(0.1, 1.5),
            "leak_rate": rng.uniform(0.05, 1.0),
            "input_scaling": rng.uniform(0.1, 1.0),
            "ridge": 10 ** rng.uniform(-8, -3),
            "seed": rng.integers(0, 10_000),
        }
        configs.append(config)

    return configs


#### Objective function

Given the following Python function, the objective becomes to maximize the AUC. The AUC is a great option in our case as it measures global separability, independently from the size of each class.

In [31]:
from reservoirpy.nodes import Reservoir, Ridge
from sklearn.metrics import roc_auc_score

def evaluate_config(config):
    reservoir = Reservoir(
        units=config["units"],
        sr=config["spectral_radius"],
        lr=config["leak_rate"],
        input_scaling=config["input_scaling"],
        seed=config["seed"],
    )

    readout = Ridge(ridge=config["ridge"])
    esn = reservoir >> readout

    # Train
    esn.fit(X_train_seq, y_train_seq)

    # Predict
    y_pred_seq = esn.run(X_test_seq)
    y_pred = np.array([yp.mean() for yp in y_pred_seq])

    # Objective: AUC
    auc = roc_auc_score(y_test_np, y_pred)

    return auc


Here, we create a function to launch the model with the radomly generated hyperparameter values. The AUC is returned to be computed.

#### Series preparation

As students, we do not own war machine computers. Hence, we choose to drastically reduce the timesteps, from 3000 to 300. We genuinely hope that the model will be able to recognize a star with an exoplanet with only 2 diracs in the serie.

In [32]:
X_train_seq_small = [x[:300] for x in X_train_seq]
X_test_seq_small  = [x[:300] for x in X_test_seq]
y_train_seq_small = [y[:300] for y in y_train_seq]

#### Parallel search

In the [reservoirpy](https://github.com/reservoirpy/reservoirpy) library, there is a function,  `parallel_research()`, which allows to speed up computation for the random search.

Unfortunately, this function is not made for the current case: it supposes that we use the `forecasting` method, a standard (X, Y) dataset and a simple scalar loss. It is not compatible with a list of sequences with one label per serie. We cannot use `forecasting` for classification, it is made for temporal prediction, on a single serie that is divided in a set of parts for training and testing.

In the current problem, we aim to classify instead of forecast. We have a set of independant series instead of one long serie, only one laber per serie and we aim to maximize the AUC instead of minimizing the MSE.

Hence, a hand-made parallel search is programmed in the following code bloc. It is rough and certainly not as clear and great than the `parallel_research()` would have done, but it follows the principle of it.

In [34]:
from joblib import Parallel, delayed

N_CONFIGS = 8
configs = sample_hyperparameters(N_CONFIGS)

results_1 = Parallel(n_jobs=-1, verbose=10)(
    delayed(evaluate_config)(cfg) for cfg in configs
)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed: 11.7min remaining: 35.0min
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed: 11.8min remaining: 19.7min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed: 12.4min remaining: 12.4min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed: 12.9min remaining:  7.7min
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed: 12.9min remaining:  4.3min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 13.0min finished


#### Results analysis

We can now analyse the first version of the results from parallel random search, to see what was the best range for each parameter.

In [35]:
results_df_1 = pd.DataFrame(configs)
results_df_1["auc"] = results_1

results_df_1 = results_df_1.sort_values("auc", ascending=False)
results_df_1

,units,spectral_radius,leak_rate,input_scaling,ridge,seed,auc
1,13,1.165596,0.796761,0.215302,1.786198e-06,9756,0.841416
0,8,0.714430,0.865668,0.727631,2.957241e-08,7739,0.740177
4,9,0.596336,0.972163,0.903809,7.796820e-05,7580,0.739823
7,17,0.281890,0.501920,0.304218,2.233932e-05,1894,0.707965
3,12,0.876419,0.110626,0.844868,1.439866e-05,2272,0.688850
2,13,1.397471,0.661672,0.840485,1.648432e-06,3707,0.684248
6,18,1.454514,0.359534,0.433414,2.227302e-06,7447,0.680000
5,16,0.753409,0.091614,0.238861,2.601625e-05,1946,0.648142


In [36]:
results_df_1.describe()

,units,spectral_radius,leak_rate,input_scaling,ridge,seed,auc
count,8.000000,8.000000,8.000000,8.000000,8.000000e+00,8.000000,8.000000
mean,13.250000,0.905008,0.544995,0.563574,1.830174e-05,5292.625000,0.716327
std,3.615443,0.406388,0.336534,0.295059,2.616839e-05,3166.088526,0.059271
min,8.000000,0.281890,0.091614,0.215302,2.957241e-08,1894.000000,0.648142
25%,11.250000,0.684906,0.297307,0.287879,1.751757e-06,2190.500000,0.683186
50%,13.000000,0.814914,0.581796,0.580522,8.312980e-06,5577.000000,0.698407
75%,16.250000,1.223564,0.813988,0.841581,2.325855e-05,7619.750000,0.739912
max,18.000000,1.454514,0.972163,0.903809,7.796820e-05,9756.000000,0.841416


**Units:** AUC values > 0.7 are concentrated around 12–17.

**Spectral radius:** values that are too low or too high result in a low AUC, so 0.6–1.4 covers the optimal range.

**Leak rate:** the best performance is around 0.35–0.85.

**Input scaling:** wide range 0.21–0.85 to capture useful dynamics.

**Ridge:** log-scale, 10^-6 → 10^-4, as observed for good results.

### Second iteration

Thanks to this first try, the values of hyperparameters will be updated and a second iteration will be made.

In [37]:
def sample_hyperparameters(n_samples, seed=42):
    rng = np.random.default_rng(seed)
    configs = []

    for _ in range(n_samples):
        config = {
            "units": rng.integers(12, 18),             
            "spectral_radius": rng.uniform(0.6, 1.4), 
            "leak_rate": rng.uniform(0.35, 0.85),    
            "input_scaling": rng.uniform(0.21, 0.85), 
            "ridge": 10 ** rng.uniform(-6, -4),      
            "seed": rng.integers(0, 10_000) 
        }
        configs.append(config)

    return configs

After setting the new values, the parallel searched is launched again. We choose to reduce the number of configuration because of the lack of great technology to run the program.

In [38]:
N_CONFIGS = 5
configs = sample_hyperparameters(N_CONFIGS)

results_2 = Parallel(n_jobs=-1, verbose=10)(
    delayed(evaluate_config)(cfg) for cfg in configs
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  8.6min remaining: 12.9min
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:  8.9min remaining:  5.9min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  8.9min finished


In [41]:
results_df_2 = pd.DataFrame(configs)
results_df_2["auc"] = results_2

results_df_2 = results_df_2.sort_values("auc", ascending=False)
results_df_2

,units,spectral_radius,leak_rate,input_scaling,ridge,seed,auc
2,15,1.341412,0.671933,0.736567,0.000008,3707,0.796460
1,15,1.208912,0.743032,0.291993,0.000008,9756,0.746195
3,14,1.043668,0.381909,0.739684,0.000018,2272,0.739115
4,12,0.883621,0.835349,0.781598,0.000036,7580,0.738761
0,12,0.951103,0.779299,0.656316,0.000002,7739,0.737345


In [42]:
results_df_2.describe()

,units,spectral_radius,leak_rate,input_scaling,ridge,seed,auc
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,13.600000,1.085743,0.682304,0.641231,0.000014,6210.800000,0.751575
std,1.516575,0.187942,0.178090,0.200424,0.000014,3104.903654,0.025326
min,12.000000,0.883621,0.381909,0.291993,0.000002,2272.000000,0.737345
25%,12.000000,0.951103,0.671933,0.656316,0.000008,3707.000000,0.738761
50%,14.000000,1.043668,0.743032,0.736567,0.000008,7580.000000,0.739115
75%,15.000000,1.208912,0.779299,0.739684,0.000018,7739.000000,0.746195
max,15.000000,1.341412,0.835349,0.781598,0.000036,9756.000000,0.796460



- The best model reached an AUC of 0.796 with a reservoir of 15 units, showing that medium-sized reservoirs are sufficient for this task.

- Spectral radii between 0.9 and 1.3 consistently produced the highest AUC, confirming the importance of operating near the edge-of-chaos regime.

- Leak rates in the range 0.6–0.8 provided the best trade-off between temporal memory and dynamical responsiveness.

- Input scaling around 0.65–0.75 ensured strong input influence without saturating the reservoir dynamics.

- Very small ridge values (≈10⁻⁶–10⁻⁵) were sufficient, indicating that only weak regularization is needed for these reservoir sizes.

These parameters can be enhanced, but for now, let us assume that they do correspond to what we could call a hypothetically optimized value.

## 8. Application on the ESN

Now, let us try to run the ESN, based on these new values.

In [44]:
from sklearn.metrics import accuracy_score, roc_auc_score
import random

units = 15
leak_rate = 0.7
input_scaling = 0.7
spectral_radius = 1.2
ridge = 1e-6
seed = random.seed(58)

reservoir = Reservoir(
    units=units,
    lr=leak_rate,
    input_scaling=input_scaling,
    sr=spectral_radius
)

readout = Ridge(ridge=ridge)

esn = reservoir >> readout

# Training
y_train_seq_np = [np.full((x.shape[0], 1), y) for x, y in zip(X_train_seq, y_train_np)] # Each timestep gets the same label (shape: timesteps x 1)
esn.fit(X_train_seq, y_train_seq_np)

# Prediction
y_pred_seq = esn.run(X_test_seq)  # returns one sequence per series
y_pred = np.array([yp.mean() for yp in y_pred_seq])  # aggregate per series
y_pred_label = (y_pred > 0.5).astype(int)

# Evaluation
accuracy = accuracy_score(y_test_np, y_pred_label)
auc = roc_auc_score(y_test_np, y_pred)

print(f"Accuracy = {round(accuracy*100, 2)}%,\nArea Under the ROC Curve = {round(auc, 2)}")

# Errors indices
errors = np.where(y_test_np != y_pred_label)[0]
print(f"Number of errors: {len(errors)} on {len(y_test_np)}")

Accuracy = 99.12%,
Area Under the ROC Curve = 0.79
Number of errors: 5 on 570


We can osberve that even though the AUC score has been enhanced, there are still 5 errors, which means a **fail**.

## Conclusion

The main problem may remain, in our case, the lack of equilibrium of dataset. Finetune hyperparameters may not be the solution for it, even though we could not experiment on high values due to our machine capacities.
